Wege um die Effizient der Schachbretterkennugn zu erhöhen:
1. Extrapolation
2. SChnittstelle einer horizontalen Linie und einer diagonalen

In [5]:
def exponential_fit(x, a, b, c):
    return a*np.exp(-b*x) + c

In [6]:
def line(p1, p2): # basis line transformation
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2): # calculate point of intersection, i can only be one and it has to be in range of picture
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y # x and y coordinates of intersection
    else:
        return False

In [7]:
def getLine(array):
    [vx, vy, x, y] = cv2.fitLine(np.array(array),cv2.DIST_L2, 0, 0.01, 0.01)
    lefty = int((-x*vy/vx) + y)
    righty = int(((image.shape[1]-x)*vy/vx)+y)
    return lefty, righty

In [8]:
import glob
import cv2
import numpy as np
from scipy.optimize import curve_fit
chess_images = glob.glob('../testFiles/emptyChessBoard/*.jpg')
# Select any index to grab an image from the list
for i, img in enumerate(chess_images):
    image = cv2.imread(img)
    im_bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    (found, corners) = cv2.findChessboardCorners(im_bw, (7, 7), None)
    if found:
        # Draw and display the corners
        cv2.drawChessboardCorners(image, (7, 7), corners, found)
        # Reshape matrix
        corners = np.reshape(corners,(7,7,2))

        # separate tuple into x and y
        #for corner in corners[0]:
        #    image = cv2.circle(image, (corner[0].astype(int),corner[1].astype(int)), radius=2, color=(0, 255, 0), thickness=-1)
        #    x.append(corner[0])
        #    y.append(corner[1])
        # Do the extrapolation through fitline
        diagonal = [corners[1][6],corners[2][5],corners[3][4],corners[4][3],corners[5][2],corners[6][1]]
        x1, y1 = getLine(corners[0])
        x2, y2 = getLine(diagonal)
        color = [0,0,255] # color hoz lines red

        cv2.line(image, (image.shape[1]-1,y1),(0,x1), color=color, thickness=1)
        cv2.line(image, (image.shape[1]-1,y2),(0,x2), color=color, thickness=1)

        L1 = line((image.shape[1]-1,y1),(0,x1))
        L2 = line((image.shape[1]-1,y2),(0,x2))

        R = intersection(L1, L2)

        image = cv2.circle(image, (int(R[0]),int(R[1])) , radius=2, color=(0, 255, 0), thickness=-1)

    result_name = 'result/ChessBoard'+str(i)+'.jpg'
    cv2.imwrite(result_name, image)